In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report
import time

In [ ]:
!pip uninstall -y qiskit qiskit-aer qiskit-terra qiskit-machine-learning qiskit-ibmq-provider qiskit-ignis qiskit-optimization

Found existing installation: qiskit 1.4.3
Uninstalling qiskit-1.4.3:
  Successfully uninstalled qiskit-1.4.3
Found existing installation: qiskit-aer 0.17.1
Uninstalling qiskit-aer-0.17.1:
  Successfully uninstalled qiskit-aer-0.17.1
Found existing installation: qiskit-terra 0.46.3
Uninstalling qiskit-terra-0.46.3:
  Successfully uninstalled qiskit-terra-0.46.3
Found existing installation: qiskit-machine-learning 0.6.1
Uninstalling qiskit-machine-learning-0.6.1:
  Successfully uninstalled qiskit-machine-learning-0.6.1


In [ ]:
!pip install "qiskit" "qiskit-aer" "qiskit_algorithms" "qiskit-machine-learning>=0.7.0"

  Using cached qiskit-2.1.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_aer-0.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.3 kB)
  Using cached qiskit_machine_learning-0.8.3-py3-none-any.whl.metadata (13 kB)
  Using cached qiskit-1.4.3-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit_aer-0.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached qiskit_machine_learning-0.8.3-py3-none-any.whl (231 kB)
Using cached qiskit-1.4.3-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.8 MB)


# Qiskit imports

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.primitives import Sampler
import time
import numpy as np

class QNNImageClassifier:
    def __init__(self, n_qubits=None, n_features=16, optimizer='SPSA', max_iter=100):
        if n_qubits is None:
            self.n_qubits = n_features
        else:
            if n_qubits < n_features:
                print(f"Warning: n_qubits ({n_qubits}) < n_features ({n_features}). Using n_features for qubit count.")
            self.n_qubits = max(n_qubits, n_features)

        self.n_features = n_features
        self.max_iter = max_iter

        from sklearn.preprocessing import StandardScaler, MinMaxScaler
        from sklearn.decomposition import PCA
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=n_features)
        self.feature_scaler = MinMaxScaler(feature_range=(0, np.pi))

        if optimizer == 'SPSA':
            self.optimizer = SPSA(maxiter=max_iter, learning_rate=0.01, perturbation=0.01)
        else:
            self.optimizer = COBYLA(maxiter=max_iter)

        self._setup_quantum_circuit()

    def _setup_quantum_circuit(self):
        self.feature_map = ZZFeatureMap(
            feature_dimension=self.n_features,
            reps=1,
            entanglement='linear'
        )

        self.ansatz = RealAmplitudes(
            num_qubits=self.n_features,
            reps=2,
            entanglement='full'
        )

        self.qc = QuantumCircuit(self.n_features)
        self.qc.compose(self.feature_map, inplace=True)
        self.qc.compose(self.ansatz, inplace=True)

        qc_with_measurements = self.qc.copy()
        qc_with_measurements.measure_all()

        self.sampler = Sampler()

        self.qnn = SamplerQNN(
            circuit=qc_with_measurements,
            input_params=self.feature_map.parameters,
            weight_params=self.ansatz.parameters,
            sampler=self.sampler
        )

        # interpretation is removed as it's deprecated
        self.classifier = NeuralNetworkClassifier(
            neural_network=self.qnn,
            optimizer=self.optimizer
        )

    def preprocess_data(self, X, y=None, fit_transform=True):
        if fit_transform:
            X_scaled = self.scaler.fit_transform(X)
            X_pca = self.pca.fit_transform(X_scaled)
            X_processed = self.feature_scaler.fit_transform(X_pca)
        else:
            X_scaled = self.scaler.transform(X)
            X_pca = self.pca.transform(X_scaled)
            X_processed = self.feature_scaler.transform(X_pca)

        if y is not None:
            y_processed = (y > y.mean()).astype(int)
            return X_processed, y_processed
        return X_processed

    def train(self, X_train, y_train, verbose=True):
        start_time = time.time()
        X_processed, y_processed = self.preprocess_data(X_train, y_train, fit_transform=True)

        if verbose:
            print(f"Training QNN with {len(X_processed)} samples...")
            print(f"Feature dimension: {self.n_features}, Qubits: {self.n_qubits}")

        self.classifier.fit(X_processed, y_processed)
        training_time = time.time() - start_time

        if verbose:
            print(f"Training completed in {training_time:.2f} seconds")

        return training_time

    def predict(self, X_test):
        X_processed = self.preprocess_data(X_test, fit_transform=False)
        return self.classifier.predict(X_processed)

    def evaluate(self, X_test, y_test, verbose=True):
        start_time = time.time()
        _, y_processed = self.preprocess_data(X_test, y_test, fit_transform=False)
        predictions = self.predict(X_test)

        from sklearn.metrics import accuracy_score, classification_report
        accuracy = accuracy_score(y_processed, predictions)
        prediction_time = time.time() - start_time

        if verbose:
            print(f"\n=== QNN Performance Report ===")
            print(f"Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
            print(f"Prediction Time: {prediction_time:.2f} seconds")
            print(f"Samples/sec: {len(X_test)/prediction_time:.2f}")
            print("\n" + classification_report(y_processed, predictions))

        return accuracy, prediction_time

def load_and_prepare_data():
    from sklearn.datasets import load_digits
    from sklearn.model_selection import train_test_split
    digits = load_digits()
    X, y = digits.data, digits.target
    y_binary = (y % 2 == 0).astype(int)

    return train_test_split(X, y_binary, test_size=0.2, random_state=42, stratify=y_binary)

def demonstrate_qnn_performance():
    print("🚀 Quantum Neural Network Image Detection Demo")
    print("=" * 50)

    X_train, X_test, y_train, y_test = load_and_prepare_data()

    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Feature dimension: {X_train.shape[1]}")

    qnn_classifier = QNNImageClassifier(
        n_features=8,
        optimizer='SPSA',
        max_iter=50
    )

    print(f"Initialized QNN with {qnn_classifier.n_qubits} qubits and {qnn_classifier.n_features} features.")
    training_time = qnn_classifier.train(X_train, y_train)
    accuracy, prediction_time = qnn_classifier.evaluate(X_test, y_test)

    print(f"\n🎯 Summary:")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Prediction Speed: {len(X_test)/prediction_time:.2f} samples/sec")
    print(f"Final Accuracy: {accuracy*100:.2f}%")

    return qnn_classifier, accuracy

def optimize_for_higher_accuracy():
    print("\n🔧 Optimizing for Higher Accuracy...")
    print("=" * 40)

    X_train, X_test, y_train, y_test = load_and_prepare_data()

    qnn_optimized = QNNImageClassifier(
        n_features=12,
        optimizer='COBYLA',
        max_iter=100
    )

    print(f"Initialized QNN with {qnn_optimized.n_qubits} qubits and {qnn_optimized.n_features} features.")
    training_time = qnn_optimized.train(X_train, y_train)
    accuracy, prediction_time = qnn_optimized.evaluate(X_test, y_test)

    return qnn_optimized, accuracy

if __name__ == "__main__":
    classifier, acc1 = demonstrate_qnn_performance()

    if acc1 < 0.75:
        print("\n Accuracy below target, trying optimized configuration...")
        classifier_opt, acc2 = optimize_for_higher_accuracy()

        if acc2 >= 0.75:
            print(" Optimized version achieved target accuracy!")
        else:
            print(" Consider increasing n_features or max_iter for better accuracy")

    print("\n QNN Image Detection Demo Complete!")

🚀 Quantum Neural Network Image Detection Demo
Training samples: 1437
Test samples: 360
Feature dimension: 64
Initialized QNN with 8 qubits and 8 features.
Training QNN with 1437 samples...
Feature dimension: 8, Qubits: 8


/tmp/ipython-input-49-2239278593.py:56: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  self.sampler = Sampler()
/tmp/ipython-input-49-2239278593.py:58: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  self.qnn = SamplerQNN(


Training completed in 1559.91 seconds

=== QNN Performance Report ===
Test Accuracy: 0.0083 (0.83%)
Prediction Time: 3.36 seconds
Samples/sec: 107.08

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       182
           1       1.00      0.02      0.03       178
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00      0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_